In [ ]:
import decimal


In [ ]:
class Account:
    def __init__(self, balance, account_number):
        self._balance = balance
        self._account_number = account_number

    @classmethod
    def create_account(cls, account_number):
        return cls(0.0, account_number)

    def deposit(self, amount):
        if amount > 0:
            self._balance += amount
        else:
            raise ValueError('Amount must be positive')

    def withdraw(self, amount):
        if amount > 0:
            self._balance -= amount
        else:
            raise ValueError('Amount must be positive')

    def get_balance(self):
        return self._balance

    def get_account_number(self):
        return self._account_number

    def __str__(self):
        return f'Account number: {self._account_number}, balance: {self._balance}'

In [ ]:
class SavingsAccount(Account):
    def __init__(self, balance, account_number, interest):
        super().__init__(balance, account_number)
        self._interest = interest

    def add_interest(self):
        self._balance += self._balance * self._interest / 100

class CurrentAccount(Account):
    def __init__(self, balance, account_number, overdraft_limit):
        super().__init__(balance, account_number)
        self._overdraft_limit = overdraft_limit

    def send_overdraft_letter(self):
        if self._balance < 0 and abs(self._balance) > self._overdraft_limit:
            print(f"Overdraft letter sent for account {self._account_number}")

class Bank:
    def __init__(self):
        self._accounts = []

    def open_account(self, account):
        self._accounts.append(account)

    def close_account(self, account_number):
        self._accounts = [acc for acc in self._accounts if acc.get_account_number() != account_number]

    def pay_dividend(self, dividend):
        for account in self._accounts:
            account.deposit(dividend)

    def update_accounts(self):
        for account in self._accounts:
            if isinstance(account, SavingsAccount):
                account.add_interest()
            elif isinstance(account, CurrentAccount):
                account.send_overdraft_letter()

    def display_accounts(self):
        for account in self._accounts:
            print(account)




In [ ]:
import unittest


In [ ]:

class TestBank(unittest.TestCase):
    def setUp(self):
        self.bank = Bank()
        self.savings_account = SavingsAccount(decimal.Decimal('1000.0'), 'SA123', decimal.Decimal('2.0'))
        self.current_account = CurrentAccount(500.0, 'CA456', -1000.0)

    def test_open_account(self):
        initial_balance = 1000.0
        account_number = 'SA123'
        self.bank.open_account(self.savings_account)
        for account in self.bank._accounts:
            if account.get_account_number() == account_number:
                self.assertEqual(account.get_balance(), initial_balance)
                break
        else:
            self.fail(f"Account with account number {account_number} not found")

    def test_update_accounts(self):
        self.bank.open_account(self.savings_account)
        self.bank.open_account(self.current_account)

        self.bank.update_accounts()

        updated_balance = self.savings_account.get_balance() * (1 + self.savings_account._interest / 100)

        threshold = decimal.Decimal('0.01')

        self.assertTrue(abs(self.savings_account.get_balance() - updated_balance) < threshold)

        import io
        from contextlib import redirect_stdout
        f = io.StringIO()
        with redirect_stdout(f):
            self.current_account.send_overdraft_letter()
        self.bank.update_accounts()
        self.assertIn("Overdraft letter sent", f.getvalue())


In [ ]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestBank)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

test_open_account (__main__.TestBank) ... ok
test_update_accounts (__main__.TestBank) ... FAIL

FAIL: test_update_accounts (__main__.TestBank)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-22-76355f708d5e>", line 31, in test_update_accounts
    self.assertTrue(abs(self.savings_account.get_balance() - updated_balance) < threshold)
AssertionError: False is not true

----------------------------------------------------------------------
Ran 2 tests in 0.011s

FAILED (failures=1)


<unittest.runner.TextTestResult run=2 errors=0 failures=1>